# 🚀 Real CUDA Fluid Simulation

This notebook actually **compiles and runs the CUDA code** from the repository!
We'll build the fluidsGL executable and run the real CUDA fluid simulation.

**Requirements:** This needs a system with:
- NVIDIA GPU
- CUDA toolkit installed
- OpenGL/GLUT development libraries

If you're on Google Colab, make sure to select GPU runtime!

In [ ]:
# First, let's check our environment
import os
import subprocess

def run_command(cmd):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        return result.returncode == 0, result.stdout, result.stderr
    except Exception as e:
        return False, "", str(e)

print("🔍 Environment Check")
print("=" * 50)

# Check if we have GPU
success, stdout, stderr = run_command("nvidia-smi")
if success:
    print("✅ GPU detected!")
    print(stdout.split('\n')[0])  # First line with driver info
else:
    print("❌ No GPU detected or nvidia-smi not available")
    print("   You'll need an NVIDIA GPU to run this efficiently")

# Check Python environment
print(f"\n🐍 Python: {os.sys.version}")
print(f"📁 Working directory: {os.getcwd()}")

print("\n" + "=" * 50)

In [ ]:
# Clone or navigate to the repository
repo_url = "https://github.com/tarun-bandi/cuda-fluid-sim.git"
repo_dir = "cuda-fluid-sim"

if os.path.exists(repo_dir):
    print(f"📁 Directory {repo_dir} already exists")
    os.chdir(repo_dir)
    !git pull  # Update if needed
else:
    print(f"📥 Cloning {repo_url}...")
    !git clone {repo_url}
    os.chdir(repo_dir)

print(f"\n📋 Repository contents:")
!ls -la

print(f"\n🔍 Source files:")
!find . -name '*.cu' -o -name '*.cpp' -o -name '*.cuh' | head -10

## 🛠️ Install Dependencies

Let's install the required tools and libraries:

In [ ]:
# Install CUDA and development tools
print("🔧 Installing dependencies...")

# Update package list
!apt-get update -qq

# Install CUDA toolkit and build tools
!apt-get install -y -qq nvidia-cuda-toolkit
!apt-get install -y -qq cmake build-essential

# Install OpenGL/GLUT for graphics
!apt-get install -y -qq freeglut3-dev libglew-dev libglu1-mesa-dev

# Install virtual framebuffer for headless OpenGL
!apt-get install -y -qq xvfb

print("\n✅ Dependencies installed!")

# Check CUDA installation
print("\n🔍 CUDA Version:")
success, stdout, stderr = run_command("nvcc --version")
if success:
    print(stdout)
else:
    print("❌ CUDA compiler not found!")
    print(stderr)

## 📝 Examine the CUDA Source Code

Let's look at the actual CUDA kernels:

In [ ]:
# Display the CUDA kernel source
kernel_file = "src/fluidsGL_kernels.cu"
main_file = "src/fluidsGL.cpp"

if os.path.exists(kernel_file):
    print("🔥 CUDA Kernel File:")
    print("=" * 50)
    with open(kernel_file, 'r') as f:
        content = f.read()
        # Show first part of the file
        lines = content.split('\n')
        for i, line in enumerate(lines[:50]):
            print(f"{i+1:3d}: {line}")
        if len(lines) > 50:
            print(f"... ({len(lines) - 50} more lines)")
else:
    print(f"❌ {kernel_file} not found!")
    print("Available files:")
    !ls -la src/

print("\n" + "=" * 50)
print(f"📊 Code Statistics:")
!wc -l src/*.cu src/*.cpp 2>/dev/null || echo "Source files not found"

## 🔨 Build the CUDA Application

Now let's compile the CUDA code with detailed error checking:

In [ ]:
# Create and enter build directory
build_dir = "build"
if os.path.exists(build_dir):
    import shutil
    shutil.rmtree(build_dir)
    
os.makedirs(build_dir)
os.chdir(build_dir)

print("🔨 Building CUDA Fluid Simulation")
print("=" * 50)

# Configure with CMake
print("📋 Step 1: CMake Configuration")
cmake_success, cmake_out, cmake_err = run_command("cmake ..")

if cmake_success:
    print("✅ CMake configuration successful!")
    if cmake_out:
        print("CMake output:")
        print(cmake_out[:1000])  # Show first 1000 chars
else:
    print("❌ CMake configuration failed!")
    print("Error output:")
    print(cmake_err)
    print("\nAttempting to create a simpler CMakeLists.txt...")
    
    # Create a simpler CMake file
    simple_cmake = '''cmake_minimum_required(VERSION 3.8)
project(cuda_fluid_sim LANGUAGES CUDA CXX)

# Find CUDA
find_package(CUDA REQUIRED)
enable_language(CUDA)

# Set CUDA flags for common architectures
set(CMAKE_CUDA_FLAGS "${CMAKE_CUDA_FLAGS} -arch=sm_35")

# Create a simple test executable
add_executable(cuda_test
    ../src/fluidsGL_kernels.cu
)

set_property(TARGET cuda_test PROPERTY CUDA_SEPARABLE_COMPILATION ON)
'''
    
    with open("CMakeLists.txt", "w") as f:
        f.write(simple_cmake)
    
    # Try again
    cmake_success, cmake_out, cmake_err = run_command("cmake .")

print("\n" + "=" * 30)

In [ ]:
# Build the project
print("🔨 Step 2: Compilation")

if cmake_success:
    build_success, build_out, build_err = run_command("make -j4")
    
    if build_success:
        print("✅ Build successful!")
        print("\n📁 Build artifacts:")
        !ls -la
        
        # Look for executables
        executables = [f for f in os.listdir('.') if os.access(f, os.X_OK) and not f.startswith('.')]
        if executables:
            print(f"\n🎯 Found executables: {', '.join(executables)}")
        else:
            print("\n❌ No executables found after build")
            
    else:
        print("❌ Build failed!")
        print("Build output:")
        print(build_out[:1000] if build_out else "No output")
        print("\nBuild errors:")
        print(build_err[:1000] if build_err else "No errors")
        
        # Try a simple CUDA test
        print("\n🔧 Trying direct CUDA compilation...")
        direct_success, direct_out, direct_err = run_command(
            "nvcc -o simple_test ../src/fluidsGL_kernels.cu -lcuda"
        )
        
        if direct_success:
            print("✅ Direct CUDA compilation successful!")
            !ls -la simple_test
        else:
            print("❌ Direct CUDA compilation also failed")
            print(direct_err[:500])
else:
    print("❌ Skipping build due to CMake failure")

print("\n" + "=" * 30)

## 🚀 Test CUDA Functionality

Let's create a simple test to verify CUDA is working:

In [ ]:
# Create a simple CUDA test program
cuda_test = '''
#include <iostream>
#include <cuda_runtime.h>

__global__ void testKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = idx * 2.0f;
    }
}

int main() {
    std::cout << "🚀 CUDA Test Program" << std::endl;
    
    // Check CUDA devices
    int deviceCount;
    cudaError_t error = cudaGetDeviceCount(&deviceCount);
    
    if (error != cudaSuccess) {
        std::cout << "❌ CUDA Error: " << cudaGetErrorString(error) << std::endl;
        return 1;
    }
    
    std::cout << "✅ Found " << deviceCount << " CUDA device(s)" << std::endl;
    
    if (deviceCount == 0) {
        std::cout << "❌ No CUDA devices available" << std::endl;
        return 1;
    }
    
    // Get device properties
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);
    std::cout << "🔥 GPU: " << prop.name << std::endl;
    std::cout << "💾 Memory: " << prop.totalGlobalMem / (1024*1024) << " MB" << std::endl;
    
    // Test kernel execution
    const int N = 1024;
    float *h_data = new float[N];
    float *d_data;
    
    cudaMalloc(&d_data, N * sizeof(float));
    
    dim3 blockSize(256);
    dim3 gridSize((N + blockSize.x - 1) / blockSize.x);
    
    testKernel<<<gridSize, blockSize>>>(d_data, N);
    
    error = cudaGetLastError();
    if (error != cudaSuccess) {
        std::cout << "❌ Kernel Error: " << cudaGetErrorString(error) << std::endl;
        return 1;
    }
    
    cudaMemcpy(h_data, d_data, N * sizeof(float), cudaMemcpyDeviceToHost);
    
    std::cout << "✅ Kernel executed successfully!" << std::endl;
    std::cout << "📊 First 10 results: ";
    for (int i = 0; i < 10; i++) {
        std::cout << h_data[i] << " ";
    }
    std::cout << std::endl;
    
    cudaFree(d_data);
    delete[] h_data;
    
    std::cout << "🎉 CUDA test completed successfully!" << std::endl;
    return 0;
}
'''

# Write the test program
with open("cuda_test.cu", "w") as f:
    f.write(cuda_test)

print("✅ Created CUDA test program")

# Compile and run the test
print("\n🔨 Compiling CUDA test...")
test_success, test_out, test_err = run_command("nvcc -o cuda_test cuda_test.cu")

if test_success:
    print("✅ CUDA test compiled successfully!")
    
    print("\n🚀 Running CUDA test...")
    run_success, run_out, run_err = run_command("./cuda_test")
    
    if run_success:
        print(run_out)
    else:
        print("❌ CUDA test execution failed:")
        print(run_err)
else:
    print("❌ CUDA test compilation failed:")
    print(test_err)

## 🎯 Running the Fluid Simulation

If the build was successful, let's try to run the main simulation:

In [ ]:
# Look for the main executable
executables = ['fluidsGL', 'cuda_test', 'simple_test']
found_exe = None

for exe in executables:
    if os.path.exists(exe) and os.access(exe, os.X_OK):
        found_exe = exe
        break

if found_exe:
    print(f"🎯 Found executable: {found_exe}")
    !ls -la {found_exe}
    
    if found_exe == 'fluidsGL':
        print("\n🚀 Running CUDA Fluid Simulation!")
        print("Note: This will try to open a GUI window. In headless mode, it may fail.")
        print("For local development, run: cd build && ./fluidsGL")
        
        # Try to run with virtual framebuffer
        print("\nAttempting to run with virtual display...")
        run_success, run_out, run_err = run_command("timeout 10s xvfb-run -a ./fluidsGL || echo 'GUI simulation needs display'")
        
        if run_out:
            print("Output:", run_out)
        if run_err:
            print("Errors:", run_err)
    else:
        print(f"\n🚀 Running {found_exe}...")
        run_success, run_out, run_err = run_command(f"./{found_exe}")
        
        if run_out:
            print(run_out)
        if run_err:
            print("Errors:", run_err)
else:
    print("❌ No executable found after build!")
    print("\n📁 Available files:")
    !ls -la
    
    print("\n🔧 Debug information:")
    if os.path.exists("CMakeCache.txt"):
        print("CMake cache exists - configuration ran")
    if os.path.exists("Makefile"):
        print("Makefile exists - ready to build")
    
    # Show build logs if they exist
    for log_file in ["CMakeFiles/CMakeOutput.log", "CMakeFiles/CMakeError.log"]:
        if os.path.exists(log_file):
            print(f"\n📋 {log_file} (last 20 lines):")
            with open(log_file, 'r') as f:
                lines = f.readlines()
                for line in lines[-20:]:
                    print(line.rstrip())

## 💻 For Local Development

To run this simulation on your local machine with proper GPU acceleration:

### Prerequisites:
1. **NVIDIA GPU** with CUDA support
2. **CUDA Toolkit** installed
3. **Development tools**: CMake, C++ compiler
4. **Graphics libraries**: OpenGL, GLUT

### Installation Commands:

**Ubuntu/Debian:**
```bash
sudo apt update
sudo apt install nvidia-cuda-toolkit cmake build-essential
sudo apt install freeglut3-dev libglew-dev libglu1-mesa-dev
```

**macOS:**
```bash
# Install CUDA toolkit from NVIDIA website
# Install Xcode command line tools
xcode-select --install
# OpenGL/GLUT should be available via system frameworks
```

### Build and Run:
```bash
git clone https://github.com/tarun-bandi/cuda-fluid-sim.git
cd cuda-fluid-sim
mkdir build && cd build
cmake ..
make -j4
./fluidsGL
```

### Controls:
- **Click and drag**: Add fluid density and heat
- **R key**: Reset simulation
- **ESC**: Exit

## 🔬 What This Demonstrates

This project showcases real **GPU-accelerated fluid dynamics** using:

1. **CUDA Kernels**: 
   - `advectKernel`: Particle advection with bilinear interpolation
   - `diffuseKernel`: Viscosity and heat diffusion
   - `addForceKernel`: Buoyancy forces
   - `projectionKernels`: Pressure projection for incompressible flow

2. **Advanced GPU Techniques**:
   - Texture memory for optimized access patterns
   - Efficient grid/block dimensions
   - Device memory management
   - Kernel synchronization

3. **Real-time Visualization**:
   - OpenGL rendering
   - Interactive mouse controls
   - Color-coded visualization of density and temperature

4. **Navier-Stokes Solver**:
   - Complete fluid dynamics simulation
   - Temperature-driven buoyancy
   - Proper boundary conditions

This is **production-quality CUDA code** that demonstrates advanced GPU programming! 🚀